In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import panel as pn
from dotenv import load_dotenv

load_dotenv()

pn.extension()


In [3]:
from scraper import get_latest_blog_posts, get_post_body

spinner = pn.indicators.LoadingSpinner(height=40)
spinner.value = False
body = pn.pane.Markdown(
    "This is a simple bot that generates blog posts, summaries, and tags. "
    "It uses the GPT-4 model to generate text, and uses DALLE-3 to generate images."
)
base_url = pn.widgets.Select(
    name="Base URL",
    options=["http://localhost:5959/blog/", "https://ericmjl.github.io/blog/"],
)
latest_posts_and_urls = get_latest_blog_posts(base_url.value)
blog_select = pn.widgets.Select(
    name="Blog Post", options={v: k for k, v in latest_posts_and_urls.items()}
)

summary_store = pn.pane.Markdown(name="Summary (hidden)")

linkedin_button = pn.widgets.Button(name="LinkedIn", button_type="primary")
twitter_button = pn.widgets.Button(name="Twitter", button_type="primary")
substack_button = pn.widgets.Button(name="Substack", button_type="primary")
summary_button = pn.widgets.Button(name="Summary", button_type="primary")
tags_button = pn.widgets.Button(name="Tags", button_type="primary")
banner_button = pn.widgets.Button(name="Banner", button_type="primary")

social_buttons = pn.Row(linkedin_button, twitter_button, substack_button)
content_buttons = pn.Row(summary_button, tags_button, banner_button)
buttons = pn.Column(social_buttons, content_buttons)

chatfeed = pn.chat.ChatFeed(name="Bot Output", height=800)
chatfeed.send(
    (
        "Hello! I'm a bot that generates blog post summaries, tags, "
        "social media posts, and banner images."
    ),
    user="BlogBot",
)
chatfeed.send(
    "Select a blog post on the left and then click on one of the blue buttons.",
    user="BlogBot",
)

control_panel = pn.Column(
    spinner, body, base_url, blog_select, buttons
)
output = pn.Column(chatfeed)

# app = pn.Column(control_panel, output)

In [7]:
# Now, define interactions between elements.
from llamabot import StructuredBot, ImageBot
from prompts import (
    compose_linkedin_post,
    compose_substack_post,
    compose_summary,
    compose_tags,
    compose_twitter_post,
    bannerbot_sysprompt,
)
import json5 as json
from models import LinkedInPost, TwitterPost, SubstackPost, Summary, Tags



bannerbot = ImageBot(size="1792x1024")


def social_button_callback(event, social_media_func):
    funcname2pydantic_model = {
        "compose_linkedin_post": LinkedInPost,
        "compose_substack_post": SubstackPost,
        "compose_twitter_post": TwitterPost,
        "compose_summary": Summary,
    }

    social_bot = StructuredBot(
        "You are an expert blogger.",
        model="gpt-4-turbo",
        pydantic_model=funcname2pydantic_model[social_media_func.__name__],
    )

    spinner.value = True
    _, body = get_post_body(blog_select.value)
    social_post = social_bot(social_media_func(body, blog_select.value))

    funcname2name = {
        "compose_linkedin_post": "LinkedIn post",
        "compose_substack_post": "Substack post",
        "compose_twitter_post": "Twitter post",
        "compose_summary": "summary",
    }

    chatfeed.send(
        f"Composing your {funcname2name[social_media_func.__name__]}! One moment...",
        user="BlogBot",
    )

    if social_media_func.__name__ == "compose_summary":
        summary_store.object = social_post.content

    if social_media_func.__name__ == "compose_linkedin_post":
        content = social_post.format_post()
    else:
        content = social_post.content

    chatfeed.send(content, user="BlogBot")
    spinner.value = False


def tag_button_callback(event):
    social_bot = StructuredBot(
        "You are an expert blogger.",
        model="gpt-4-turbo",
        pydantic_model=Tags,
    )
    spinner.value = True
    _, body = get_post_body(blog_select.value)
    chatfeed.send(f"Composing your tags! One moment...", user="BlogBot")
    tags = social_bot(compose_tags(body))
    chatfeed.send("I have your tags! Here they are:", user="BlogBot")
    chatfeed.send("\n".join(tags.content), user="BlogBot")
    spinner.value = False


def banner_button_callback(event):
    spinner.value = True
    _, body = get_post_body(blog_select.value)

    summary = ""
    if not summary_store.object:
        social_bot = StructuredBot(
            "You are an expert blogger.",
            model="gpt-4-turbo",
            pydantic_model=Summary,
        )
        social_post = social_bot(compose_summary(body, blog_select.value))
        chatfeed.send(
            f"I need a summary to use for image generation. Please wait one moment while I compose it...",
            user="BlogBot",
        )
        message = chatfeed.send("", user="BlogBot")
        received_message = ""
        for token in social_post.content:
            message = chatfeed.stream(token, user="BlogBot", message=message)
            received_message += token
        summary_store.object = social_post.content
    summary = summary_store.object

    chatfeed.send(f"Composing your banner! One moment...", user="BlogBot")
    for i in range(1):
        chatfeed.send(f"Generating image #{i+1}...", user="BlogBot")

        banner_url = bannerbot(bannerbot_sysprompt() + summary, return_url=True)
        print(banner_url)
        # image = pn.pane.Image(banner_url, height=448, width=256)
        chatfeed.send(f"Here is banner image #{i+1}", user="BlogBot")
        # chatfeed.send(f"Your banner URL: {banner_url}", user="BlogBot")
        chatfeed.send(banner_url, user="BlogBot")
        chatfeed.send(
            f"And [here]({banner_url}) is the raw URL to the image.", user="BlogBot"
        )
    spinner.value = False


def update_blog_select(target, event):
    latest_posts_and_urls = get_latest_blog_posts(event.obj.value)
    target.options = {v: k for k, v in latest_posts_and_urls.items()}


# Define reactive callbacks.
# Change list of available blog posts based on base URL.
base_url.link(blog_select, callbacks={"value": update_blog_select})
linkedin_button.on_click(
    lambda event: social_button_callback(event, compose_linkedin_post)
)
twitter_button.on_click(
    lambda event: social_button_callback(event, compose_twitter_post)
)
substack_button.on_click(
    lambda event: social_button_callback(event, compose_substack_post)
)
summary_button.on_click(lambda event: social_button_callback(event, compose_summary))
tags_button.on_click(tag_button_callback)
banner_button.on_click(banner_button_callback)



# Now apply template
template = pn.template.BootstrapTemplate(
    title="Eric's Blog Bot",
)

template.sidebar.append(control_panel)
template.main.append(output)

template.servable()

BootstrapTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] BootstrapTemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [nav-5221162896] Column
        [0] LoadingSpinner(height=40)
        [1] Markdown(str)
        [2] Select(name='Base URL', options=['http://localhost:5959/bl...], value='http://localhost:5959/blo...)
        [3] Select(name='Blog Post', options={'LlamaBot now has Structu...}, value='http://localhost:5959/blo...)
        [4] Column
            [0] Row
                [0] Button(button_type='primary', name='LinkedIn')
                [1] Button(button_type='primary', name='Twitter')
                [2] Button(button_type='primary', name='Substack')
            [1] Row
                [0] Button(button_type='primary', name='Summary')
                [1] Button(button_type='primary', name='Tags')
                [2] Button(button_type='primary', name='Banner')
    [main-5222494864] Column
        [0] ChatFeed(_placeholder=ChatMessage, height=800, name='Bot Output', sizing_mode='stretch_width')
            [0] ChatMessage(str, reaction_icons=ChatReactionIcons, timestamp=datetime.datetime(2024, ..., user='BlogBot')
            [1] ChatMessage(str, reaction_icons=ChatReactionIcons, timestamp=datetime.datetime(2024, ..., user='BlogBot')